<a id="TableOfContents"></a>
# Table of Contents:
<li><a href="#Q1">Question 1</li></a>
<li><a href="#Q2">Question 2</li></a>
<li><a href="#Q3">Question 3</li></a>
<li><a href="#Q4">Question 4</li></a>
<li><a href="#Q5">Question 5</li></a>

In [25]:
import pandas as pd
import sklearn.metrics
from sklearn.metrics import confusion_matrix

<a id="Q1"></a>
### 1. Create a new file named model_evaluation.py or model_evaluation.ipynb for these exercises.
<li><a href="#TableOfContents">Table of Contents</li></a>

COMPLETED

<a id="Q2"></a>
### 2. Given the following confusion matrix, evaluate (by hand) the model's performance.
<li><a href="#TableOfContents">Table of Contents</li></a>

|               | pred dog   | pred cat   |
|:------------  |-----------:|-----------:|
| actual dog    |         46 |         7  |
| actual cat    |         13 |         34 |


##### 2a. In the context of this problem, what is a false positive?

- predicted dog, actual cat

##### 2b. In the context of this problem, what is a false negative?

- Predicted cat, actual dog

##### 2c. How would you describe this model?

It's rainin' doggos 'un kitties

<a id="Q3"></a>
### 3. You are working as a datascientist working for Codeup Cody Creator (C3 for short), a rubber-duck manufacturing plant.
<li><a href="#TableOfContents">Table of Contents</li></a>

Unfortunately, some of the rubber ducks that are produced will have defects. Your team has built several models that try to predict those defects, and the data from their predictions <a href="https://ds.codeup.com/data/c3.csv">can be found here.</a>

Use the predictions dataset and pandas to help answer the following questions:

In [57]:
c3 = pd.read_csv('c3.csv')
c3.sample()

,actual,model1,model2,model3
11,No Defect,No Defect,No Defect,No Defect


##### 3a. An internal team wants to investigate the cause of the manufacturing defects. They tell you that they want to identify as many of the ducks that have a defect as possible. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

Assuming 'No Defect' is desired outcome...

I want a model to predict the most defects...

I want to use specificity:

TN / (FP + TN)

- TN df[col][1, 1]
    - Predicted Defect
    - Actual Defect
- FP df[col][1, 0]
    - Predicted No Defect
    - Actual Defect

In [61]:
for col in c3:
    matrix = confusion_matrix(c3.actual, c3[col], labels=('No Defect', 'Defect'))
    result = (matrix[1, 1] / (matrix[1, 0] + matrix[1, 1]))
    print(f'\033[32m{col}:\033[0m {result}\n')

actual: 1.0

model1: 0.5

model2: 0.5625

model3: 0.8125



model3 is the best model to predict defects

##### 3b. Recently several stories in the local news have come out highlighting customers who received a rubber duck with a defect, and portraying C3 in a bad light. The PR team has decided to launch a program that gives customers with a defective duck a vacation to Hawaii. They need you to predict which ducks will have defects, but tell you the really don't want to accidentally give out a vacation package when the duck really doesn't have a defect. Which evaluation metric would be appropriate here? Which model would be the best fit for this use case?

Assuming 'No Defect' is the desired outcome...

I want the model with the best prediction to actual...

I want to use accuracy:

(TP + TN) / (TP + FP + FN + TN)

In [64]:
for col in c3:
    ratio = (c3[col] == c3['actual']).mean()
    print(f'\033[32m{col}:\033[0m {ratio}\n')

actual: 1.0

model1: 0.95

model2: 0.56

model3: 0.555



model1 is the best to predict correct outcomes

<a id="Q4"></a>
### 4. You are working as a data scientist for Gives You Paws ™, a subscription based service that shows you cute pictures of dogs or cats (or both for an additional fee).
<li><a href="#TableOfContents">Table of Contents</li></a>

At Gives You Paws, anyone can upload pictures of their cats or dogs. The photos are then put through a two step process. First an automated algorithm tags pictures as either a cat or a dog (Phase I). Next, the photos that have been initially identified are put through another round of review, possibly with some human oversight, before being presented to the users (Phase II).

Several models have already been developed with the data, and <a href="https://ds.codeup.com/data/gives_you_paws.csv">you can find their results here.</a>

Given this dataset, use pandas to create a baseline model (i.e. a model that just predicts the most common class) and answer the following questions:

##### 4a. In terms of accuracy, how do the various models compare to the baseline model? Are any of the models better than the baseline?

##### 4b. Suppose you are working on a team that solely deals with dog pictures. Which of these models would you recommend?

##### 4c. Suppose you are working on a team that solely deals with cat pictures. Which of these models would you recommend?

<a id="Q5"></a>
### 5. Follow the links below to read the documentation about each function, then apply those functions to the data from the previous problem.
<li><a href="#TableOfContents">Table of Contents</li></a>

##### 5a. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html">sklearn.metrics.accuracy_score</a>

##### 5b. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_score.html">sklearn.metrics.precision_score</a>

##### 5c. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.recall_score.html">sklearn.metrics.recall_score</a>

##### 5d. <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html">sklearn.metrics.classification_report</a>